In [1]:
# --- 0. Gerekli Kütüphanelerin Kurulumu (Fabrika Sıfırlaması Sonrası) ---
print("Installing required libraries after factory reset...")

# Gerekli PyTorch ve CUDA sürümünü belirtin (Colab genelde cu118 veya cu121 kullanır, kontrol edilebilir)
# !nvidia-smi # komutuyla CUDA sürümünü görebilirsiniz
PYTORCH_CUDA_INDEX = "cu118" # veya "cu121" vb. Colab'ın CUDA sürümüne göre

# Temel ML Kütüphaneleri (Torch ailesi + Transformers ve hızlandırıcılar)
!pip install -qU torch torchvision torchaudio --index-url https://download.pytorch.org/whl/{PYTORCH_CUDA_INDEX}
!pip install -qU transformers accelerate bitsandbytes sentencepiece

# LangChain, Veri İşleme ve Arayüz Kütüphaneleri
!pip install -qU langchain langchain-huggingface pandas gradio huggingface_hub

print("\nLibrary installation complete.")

# Kurulum sonrası kütüphanelerin yüklendiğini ve import edilebildiğini doğrulamak için
try:
    import torch
    import torchvision
    import transformers
    import accelerate
    import langchain
    import gradio
    print(f"Torch version: {torch.__version__}")
    print(f"Torchvision version: {torchvision.__version__}")
    print(f"Transformers version: {transformers.__version__}")
    print(f"Accelerate version: {accelerate.__version__}")
    print(f"Langchain version: {langchain.__version__}")
    print(f"Gradio version: {gradio.__version__}")
    print("\nCore libraries imported successfully!")
except ImportError as e:
    print(f"\nERROR: Failed to import a library after installation: {e}")
except Exception as e:
    print(f"\nERROR: An unexpected error occurred during import check: {e}")

Installing required libraries after factory reset...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 87.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 121.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 955.6/955.6 M

In [ ]:
import torch
import transformers
import torchvision
import os
import pandas as pd
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import gradio as gr
from huggingface_hub import login
import traceback


# --- 1. Yapılandırma ---
HF_TOKEN = ""
LLM_MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"

CALISAN_BILGILERI_PATH = '/content/drive/MyDrive/CalısanBilgileri.csv'


try:
    login(token=HF_TOKEN)
    print("Hugging Face login successful (required for downloading gated models like Llama).")
except Exception as e:
    print(f"Hugging Face login failed: {e}")
    print("WARNING: Model download might fail if the model requires authentication.")
    print("-> Ensure your HF_TOKEN is correct and you have accepted the model terms on Hugging Face Hub.")

# --- 3. Yerel LLM Başlatma ---
llm = None
model = None
tokenizer = None
pipe = None

try:
    print(f"\n--- Initializing LOCAL LLM: '{LLM_MODEL_ID}' ---")
    # Cihaz Seçimi (GPU varsa GPU, yoksa CPU)
    if torch.cuda.is_available():
        selected_device = "cuda"
        print(f"GPU Detected: {torch.cuda.get_device_name(0)}")
        # float16 VRAM kullanımını azaltır, T4 gibi kartlar için uygun
        selected_torch_dtype = torch.float16
        print("Using torch.float16 for model.")
    else:
        selected_device = "cpu"
        selected_torch_dtype = torch.float32 # CPU'da float16 her zaman desteklenmez
        print("No GPU detected. LLM will run on CPU (This can be VERY SLOW!).")
        print("Using torch.float32 for model.")

    # Tokenizer Yükleme
    print("\nLoading Tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL_ID, token=HF_TOKEN)
    print("Tokenizer loaded successfully.")

    # Model Yükleme
    print(f"\nLoading Model '{LLM_MODEL_ID}'...")
    print("This might take a while and require significant RAM/VRAM and Disk Space.")
    # device_map="auto": Modeli otomatik olarak GPU/CPU'ya dağıtır (accelerate kütüphanesi gereklidir)
    # torch_dtype=torch.float16: GPU VRAM kullanımını azaltır
    model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL_ID,
        torch_dtype=selected_torch_dtype,
        device_map="auto", # Otomatik cihaz dağıtımı için accelerate gerekir
        token=HF_TOKEN,    # Llama gibi erişim kısıtlı modeller için gerekli
        # --- Bellek Optimizasyonları (Gerekirse Aktif Edin) ---
        # load_in_8bit=True, # 8-bit yükleme (daha az VRAM, biraz performans kaybı)
        # load_in_4bit=True, # 4-bit yükleme (en az VRAM, daha fazla performans kaybı, bitsandbytes gerekir)
        # bnb_4bit_quant_type="nf4", # 4-bit için kuantizasyon türü
        # bnb_4bit_compute_dtype=torch.bfloat16 # 4-bit için hesaplama türü
    )
    print("Model loaded successfully.")

    # Transformers Pipeline Oluşturma
    print("\nCreating text-generation pipeline...")
    # device_map="auto" kullanıldığı için pipeline'a device belirtmeye gerek yok
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        # torch_dtype=selected_torch_dtype, # Model zaten bu dtype ile yüklendi
        # device=selected_device, # device_map="auto" bunu yönetir
        max_new_tokens=256, # Üretilecek maksimum token sayısı
        temperature=0.6,    # Rastgelelik seviyesi (düşük = daha tutarlı)
        do_sample=True,     # Temperature kullanmak için True olmalı
        repetition_penalty=1.1 # Tekrarları azaltmak için
    )
    print("Pipeline created successfully.")

    # LangChain için Pipeline'ı Sarmalama
    llm = HuggingFacePipeline(pipeline=pipe)
    print(f"\nLOCAL LLM '{LLM_MODEL_ID}' initialized successfully via HuggingFacePipeline.")

except ImportError as e:
     print(f"\n--- ERROR initializing local LLM: Missing library -> {e} ---")
     print("Please ensure 'transformers', 'torch', 'accelerate' are installed.")
     print("If using 4-bit loading, also install 'bitsandbytes'.")
     print("In Colab, run the !pip install cell at the beginning.")
     traceback.print_exc()
     llm = None
except RuntimeError as e: # Özellikle OutOfMemoryError için
    print(f"\n--- ERROR initializing local LLM: Runtime Error -> {e} ---")
    if "out of memory" in str(e).lower():
        print("\nCUDA Out Of Memory Error detected!")
        print("The model is likely too large for your available GPU VRAM.")
        print("Troubleshooting steps:")
        print("1. Restart your runtime/kernel to free up memory.")
        print("2. Try enabling 8-bit or 4-bit loading in the 'AutoModelForCausalLM.from_pretrained' call (see commented lines).")
        print("   - Requires 'bitsandbytes' library: !pip install bitsandbytes")
        print("3. Use a smaller model (e.g., a 4B or smaller version if available).")
        print("4. If running on Colab, check if you have a High-RAM runtime enabled (if available).")
    traceback.print_exc()
    llm = None
except Exception as e:
    print(f"\n--- ERROR initializing local LLM: An unexpected error occurred -> {e} ---")
    traceback.print_exc()
    print("\nPossible causes:")
    print("- Ensure your HF_TOKEN is valid and you have accepted the model's terms on Hugging Face Hub.")
    print("- Check if you have enough disk space for the model download.")
    print("- Verify your internet connection for the download.")
    llm = None

# --- 4. Çalışan Verisi Yükleme ---
calisanlar_df = None
print(f"\n--- Loading Employee Data from: {CALISAN_BILGILERI_PATH} ---")
try:
    calisanlar_df = pd.read_csv(CALISAN_BILGILERI_PATH)
    print("Employee CSV data loaded successfully.")
    # print("First 5 rows of the data:\n", calisanlar_df.head()) # Veriyi kontrol etmek için
    # print("\nColumns in the CSV:", calisanlar_df.columns.tolist()) # Sütun adlarını kontrol et
except FileNotFoundError:
    print(f"ERROR: Employee CSV file not found at: {CALISAN_BILGILERI_PATH}")
    print("Troubleshooting:")
    print("- Is the file path correct?")
    print("- If using Google Colab, did you mount Google Drive correctly (see the beginning of the script)?")
    print("- Is the file actually present in the specified Google Drive folder?")
    calisanlar_df = None
except Exception as e:
    print(f"ERROR loading employee CSV data: {e}")
    traceback.print_exc()
    calisanlar_df = None

# --- 5. Chatbot Fonksiyonu ---

# Anahtar Kelime Eşleştirme Sözlüğü (CSV'deki sütun adlarıyla eşleşmeli)
KEYWORD_TO_BILGI_MAPPING = {
    # Anahtar (Dropdown/Kod içi kullanım) : [Kullanıcıdan gelebilecek anahtar kelimeler] -> CSV Sütun Adı (get için)
    "Maaş": (["maaş", "ücret", "kazanç", "bordro", "salary", "ne kadar alıyorum"], "Maaş"),
    "İzin Durumu": (["izin", "yıllık izin", "kalan izin", "tatil", "leave", "vacation"], "İzin Günleri"), # CSV'de 'İzin Günleri' olduğunu varsayıyoruz
    "Departman": (["departman", "bölüm", "department"], "Departman"),
    "Pozisyon": (["pozisyon", "görev", "title", "rol"], "Pozisyon"),
    "Yönetici": (["yönetici", "amir", "müdür", "manager", "supervisor", "kime bağlıyım"], "Yönetici"),
    "İşe Başlama Tarihi": (["işe başlama", "başlangıç tarihi", "ne zaman başladım", "start date"], "İşe Başlama Tarihi"),
    "Telefon": (["telefon", "numara", "tel", "gsm", "phone"], "Telefon"),
    "Email": (["email", "e-posta", "mail"], "Email"),
    "Performans Değerlendirme": (["performans", "değerlendirme", "puan", "performance review"], "Performans Değerlendirme"), # CSV'de bu sütun varsa
    "Hastalık İzni": (["hastalık izni", "raporlu", "rapor", "sick leave"], "Hastalık İzni") # CSV'de bu sütun varsa
}

def chatbot(sicil_no_str, bilgi_turu_dropdown, user_msg):
    """
    Ana chatbot fonksiyonu. Sicil no, dropdown seçimi ve kullanıcı mesajını alır,
    CSV veya yerel LLM kullanarak yanıt üretir.
    """
    print("-" * 30)
    print(f"Received Input: Sicil='{sicil_no_str}', Dropdown='{bilgi_turu_dropdown}', Message='{user_msg}'")

    # --- Sicil No Kontrolü ---
    if not sicil_no_str or not sicil_no_str.strip():
        return "⚠ Lütfen Çalışan Sicil Numarasını girin."
    try:
        sicil_no = int(sicil_no_str)
    except ValueError:
        return "⚠ Geçersiz sicil numarası! Lütfen yalnızca rakam girin."

    # --- Çalışan Bulma (CSV Varsa) ---
    calisan = None
    ad_soyad = f"Çalışan {sicil_no}" # Varsayılan
    calisan_pozisyon = "N/A"        # Varsayılan
    if calisanlar_df is None:
        print("WARNING: Employee data (CSV) is not loaded. Cannot fetch personal info.")
        # CSV yoksa, kişisel bilgi döndürülemez ama genel sorular yine de sorulabilir.
        # Bu durumda ad/soyad/pozisyon bilinmeyecek.
    else:
        try:
            # 'Sicil No' sütununun CSV'de olduğundan emin olun
            if 'Sicil No' not in calisanlar_df.columns:
                 print("ERROR: 'Sicil No' column not found in the CSV file!")
                 return "⚠ Sistem Hatası: Çalışan veritabanında 'Sicil No' sütunu bulunamadı. Lütfen yönetici ile iletişime geçin."

            calisan_df_filtered = calisanlar_df[calisanlar_df['Sicil No'] == sicil_no]
            if not calisan_df_filtered.empty:
                calisan = calisan_df_filtered.iloc[0] # Çalışan verisini (satırı) al
                # Ad/Soyad ve Pozisyon bilgilerini al (CSV'de bu sütunlar varsa)
                temp_ad = calisan.get('Ad', '') # .get() sütun yoksa hata vermez
                temp_soyad = calisan.get('Soyad', '')
                if temp_ad or temp_soyad:
                    ad_soyad = f"{temp_ad} {temp_soyad}".strip()
                calisan_pozisyon = calisan.get('Pozisyon', 'N/A')
                print(f"Employee Found: Sicil={sicil_no}, Name={ad_soyad}, Position={calisan_pozisyon}")
            else:
                print(f"Employee Not Found: Sicil={sicil_no}")
                # Sicil bulunamadıysa, ne kişisel bilgi verilebilir ne de LLM'e anlamlı prompt gönderilebilir.
                return f"⚠ {sicil_no} numaralı sicil sistemde bulunamadı! Lütfen geçerli bir numara girin."
        except Exception as e:
            print(f"Error during employee lookup: {e}")
            traceback.print_exc()
            return "⚠ Çalışan bilgileri aranırken bir hata oluştu. Lütfen tekrar deneyin veya yönetici ile iletişime geçin."

    # --- Yanıt Mantığı ---

    # Öncelik: Kullanıcının yazdığı mesaj (user_msg)
    if user_msg and user_msg.strip():
        processed_msg = user_msg.lower().strip()
        print(f"Processing User Message: '{processed_msg}'")

        # 1a. Selamlama Kontrolü
        greetings = ["selam", "merhaba", "mrb", "slm", "hi", "hello", "günaydın", "iyi günler", "iyi akşamlar", "naber", "hey"]
        if processed_msg in greetings:
             return f"Merhaba {ad_soyad}! Size nasıl yardımcı olabilirim?"

        # 1b. Kişisel Bilgi Anahtar Kelimesi Kontrolü
        found_personal_keyword = False
        if calisan is not None: # Sadece çalışan bulunduysa kişisel bilgi verebiliriz
            for bilgi_key, (keywords, csv_column) in KEYWORD_TO_BILGI_MAPPING.items():
                if any(keyword in processed_msg for keyword in keywords):
                    print(f"DEBUG: Keyword match for '{bilgi_key}' found in message.")
                    found_personal_keyword = True
                    # İlgili sütun CSV'de var mı ve değeri dolu mu kontrol et
                    if csv_column in calisan and pd.notna(calisan[csv_column]):
                        value = calisan[csv_column]
                        # Yanıt formatlama (örnekler)
                        if bilgi_key == "Maaş":
                            return f"💰 {ad_soyad}, maaş bilginiz: {value} TL."
                        elif bilgi_key == "İzin Durumu":
                            return f"📆 {ad_soyad}, kalan izin günleriniz: {value} gün."
                        elif bilgi_key == "Hastalık İzni":
                             return f"🤒 {ad_soyad}, kalan hastalık izniniz: {value} gün."
                        # Diğer bilgiler için genel format
                        else:
                            # Emoji eklemek isteğe bağlı
                            emojis = {"Departman": "🏢", "Pozisyon": "📌", "Yönetici": "👨‍💼",
                                      "İşe Başlama Tarihi": "📅", "Telefon": "📞", "Email": "📧",
                                      "Performans Değerlendirme": "📊"}
                            prefix = emojis.get(bilgi_key, "ℹ️") # Bulamazsa info emojisi
                            return f"{prefix} {ad_soyad}, {bilgi_key.lower()} bilginiz: {value}."
                    else:
                        print(f"DEBUG: Column '{csv_column}' not found or empty in CSV for employee {sicil_no}.")
                        return f"⚠ Üzgünüm {ad_soyad}, '{bilgi_key}' bilginiz sistemde bulunamadı veya eksik."
            # Eğer döngü bitti ve kişisel anahtar kelime bulunduysa ama return olmadıysa (yukarıdaki else'e düştüyse)
            if found_personal_keyword:
                 # Bu duruma normalde yukarıdaki else düşmeli ama bir güvenlik önlemi
                 return f"⚠ Üzgünüm {ad_soyad}, istediğiniz bilgi sistemde bulunamadı."

        # 1c. Kişisel bilgi değilse ve selamlama değilse -> Genel LLM'e Sor
        if not found_personal_keyword:
            if llm: # LLM başarıyla yüklendi mi?
                try:
                    print(f"DEBUG: Sending general query to LOCAL LLM: '{user_msg}'")

                    # Llama 3 Instruct formatına uygun prompt oluşturma
                    # System prompt'u genel talimatları içerir
                    # User prompt'u kullanıcının sorusunu içerir
                    # Assistant prompt'unun başlangıcı LLM'in yanıtlamasını tetikler
                    system_prompt = f"Sen Kipaş şirketinin yardımsever bir İK asistanısın. Çalışan '{ad_soyad}' (Pozisyon: {calisan_pozisyon}) sana bir soru soruyor. Sorusuna kısa, profesyonel, nazik ve öz bir şekilde cevap ver. Eğer soru şirket politikaları, genel İK süreçleri veya işle ilgili genel konular hakkındaysa cevap ver. Çalışanın kişisel verilerini (maaş, izin vb.) ifşa etme, bu tür bilgiler istendiğinde 'Bu bilgiye erişimim yok, lütfen ilgili menüden kontrol edin veya İK departmanı ile iletişime geçin.' gibi bir yanıt ver."
                    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>
{user_msg}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
                    print(f"DEBUG: Prompt sent to LLM:\n{prompt}")

                    # LLM'i çağır
                    response = llm.invoke(prompt)
                    print(f"DEBUG: Raw response from LOCAL LLM:\n{response}")

                    # --- Yanıt Temizleme ---
                    # 1. Yanıttan girdi prompt'unu çıkar (pipeline bazen tekrarlayabilir)
                    # Yanıtın assistant kısmından başlaması beklenir.
                    answer = response.strip()
                    if prompt in answer: # Eğer tüm prompt tekrarlanmışsa
                        cleaned_answer = answer.split("<|start_header_id|>assistant<|end_header_id|>")[-1].strip()
                    elif "<|start_header_id|>assistant<|end_header_id|>" in answer: # Sadece assistant kısmı varsa
                         cleaned_answer = answer.split("<|start_header_id|>assistant<|end_header_id|>")[-1].strip()
                    else: # Beklenmedik format, ham yanıtı al
                         cleaned_answer = answer

                    # 2. Llama 3'ün özel bitiş token'ını (<|eot_id|>) temizle (yanıtın sonunda olabilir)
                    if cleaned_answer.endswith("<|eot_id|>"):
                        cleaned_answer = cleaned_answer[:-len("<|eot_id|>")].strip()

                    print(f"DEBUG: Cleaned LLM response:\n{cleaned_answer}")

                    # 3. Yanıtın anlamlı olup olmadığını kontrol et
                    if not cleaned_answer or len(cleaned_answer) < 10:
                         print(f"DEBUG: LLM response too short or potentially irrelevant after cleaning.")
                         return f"Merhaba {ad_soyad}. '{user_msg}' sorunuzu aldım. Ancak şu an için bu konuda sistemden net bir yanıt alamadım. Lütfen sorunuzun açık olduğundan emin olun veya İK departmanıyla iletişime geçin."
                    else:
                        return cleaned_answer # Temizlenmiş yanıtı döndür

                except Exception as e:
                    print(f"ERROR during local LLM inference: {e}")
                    traceback.print_exc()
                    return f"Üzgünüm {ad_soyad}, genel sorunuzu ('{user_msg}') yanıtlarken yerel modelde bir hata oluştu. Lütfen sistem yöneticisine bildirin."
            else: # LLM başlatılamamışsa
                 return f"Merhaba {ad_soyad}. Şu anda genel sorularınızı ('{user_msg}') yanıtlayamıyorum çünkü yapay zeka modeli başlatılamadı."

    # 2. Kullanıcı mesajı yoksa, Dropdown seçimine bak
    elif bilgi_turu_dropdown:
        print(f"Processing Dropdown Selection: '{bilgi_turu_dropdown}'")
        if calisan is None:
             return f"⚠ Üzgünüm, çalışan bilgileri yüklenemediği için '{bilgi_turu_dropdown}' bilgisini gösteremiyorum."

        if bilgi_turu_dropdown in KEYWORD_TO_BILGI_MAPPING:
            keywords, csv_column = KEYWORD_TO_BILGI_MAPPING[bilgi_turu_dropdown]
            if csv_column in calisan and pd.notna(calisan[csv_column]):
                value = calisan[csv_column]
                # Yanıt formatlama (yukarıdakiyle aynı)
                if bilgi_turu_dropdown == "Maaş":
                    return f"💰 {ad_soyad}, maaş bilginiz: {value} TL."
                elif bilgi_turu_dropdown == "İzin Durumu":
                    return f"📆 {ad_soyad}, kalan izin günleriniz: {value} gün."
                elif bilgi_turu_dropdown == "Hastalık İzni":
                     return f"🤒 {ad_soyad}, kalan hastalık izniniz: {value} gün."
                else:
                    emojis = {"Departman": "🏢", "Pozisyon": "📌", "Yönetici": "👨‍💼",
                              "İşe Başlama Tarihi": "📅", "Telefon": "📞", "Email": "📧",
                              "Performans Değerlendirme": "📊"}
                    prefix = emojis.get(bilgi_turu_dropdown, "ℹ️")
                    return f"{prefix} {ad_soyad}, {bilgi_turu_dropdown.lower()} bilginiz: {value}."
            else:
                print(f"DEBUG: Column '{csv_column}' not found or empty in CSV for dropdown selection.")
                return f"⚠ Üzgünüm {ad_soyad}, '{bilgi_turu_dropdown}' bilginiz sistemde bulunamadı veya eksik."
        else:
            # Bu durum normalde olmamalı çünkü dropdown seçenekleri mapping'den geliyor.
            print(f"ERROR: Invalid dropdown value received: {bilgi_turu_dropdown}")
            return f"⚠ Geçersiz bilgi türü seçimi: {bilgi_turu_dropdown}"

    # 3. Ne mesaj ne de dropdown seçimi varsa (sadece sicil no girilmişse)
    else:
        print("No specific question or dropdown selected.")
        return f"Merhaba {ad_soyad}! Lütfen öğrenmek istediğiniz bilgi türünü seçin veya genel sorunuzu aşağıdaki kutucuğa yazın."


# --- 6. Gradio Arayüzü ---

STATIK_BILGI_TURLERI = list(KEYWORD_TO_BILGI_MAPPING.keys()) # Dropdown için anahtarları al

# Sadece LLM başarıyla yüklendiyse ve CSV okunduysa arayüzü başlat
if llm and calisanlar_df is not None:
    print("\n" + "="*40)
    print(" SUCCESS: LLM and Employee Data Loaded ")
    print("--- Launching Gradio Interface (Local LLM Powered) ---")
    print("="*40 + "\n")

    gui = gr.Interface(
        fn=chatbot,
        inputs=[
            gr.Textbox(label="📌 Çalışan Sicil Numarası",
                       placeholder="Sicil numaranızı girin...",
                       info="Lütfen sadece rakam girin."),
            gr.Dropdown(
                choices=STATIK_BILGI_TURLERI,
                label="📋 Hızlı Bilgi Seçimi ",
                info="Sık sorulan kişisel bilgiler için buradan seçim yapabilirsiniz."
            ),
            gr.Textbox(
                label="✍️ Genel Sorunuz veya Özel Bilgi İsteğiniz",
                placeholder="Örn: 'Maaşım ne kadar?', 'Kalan izinlerimi öğrenmek istiyorum', 'Şirket araç politikası nedir?', 'Performans değerlendirme ne zaman?'",
                info="Buraya yazarsanız, yukarıdaki 'Hızlı Bilgi Seçimi' göz ardı edilir. Hem kişisel bilgilerinizi (maaş, izin vb.) hem de genel İK sorularınızı buradan sorabilirsiniz."
                )
        ],
        outputs=gr.Textbox(label="🤖 Kipaş İK Asistanı Yanıtı", lines=10), # Yanıt alanı daha geniş
        title="💼 Kipaş İK Asistanı (Yerel Yapay Zeka Destekli)",
        description=(
            "👋 Merhaba! Ben Kipaş İK Asistanınız.\n"
            "1. **Sicil Numaranızı** girin.\n"
            "2. **Kişisel Bilgiler İçin:** Ya 'Hızlı Bilgi Seçimi' menüsünü kullanın ya da sorunuzu aşağıdaki metin kutusuna yazın (örn: 'maaşım nedir?').\n"
            "3. **Genel İK Soruları İçin:** Sorunuzu aşağıdaki metin kutusuna yazın (örn: 'yıllık izin prosedürü nasıl işler?').\n"
            "   *(Yanıtlar şirket verileri ve yerel yapay zeka modeli kullanılarak üretilmektedir.)*"
        ),
        allow_flagging='never', # Bayrakla işaretleme özelliğini kapat
        theme=gr.themes.Soft(primary_hue="blue", secondary_hue="indigo"), # Biraz renk
        examples=[
            # Sicil No, Dropdown, Mesaj
            ["82384", None, "maaşımı öğrenebilir miyim?"],
            ["99328", "İzin Durumu", None], # Sadece dropdown kullanımı
            ["90392", None, "Yöneticim kim?"],
            ["32614", None, "Performans değerlendirme süreci nasıl işliyor?"], # Genel İK sorusu örneği
            ["82384", None, "Şirketteki sosyal haklar nelerdir?"], # Başka bir genel soru
            ["82384", None, "selam"], # Selamlama
            ["19200", None, "maaşım"], # Geçersiz sicil örneği (varsayımsal)
            ["88001", "Departman", "Ayrıca telefon numaram nedir?"], # Hem dropdown hem mesaj (mesaj öncelikli olacak)
        ]
    )

    print("Gradio Interface is ready.")
    print("Access the interface using the public URL provided below (if running in Colab or using share=True).")
    # share=True: Colab veya başka bir ağdan erişim için herkese açık bir URL oluşturur.
    # debug=True: Gradio konsolunda daha fazla hata ayıklama bilgisi gösterir.
    gui.launch(share=True, debug=True)

else:
     # Başlatma başarısız olduysa detaylı bilgi ver
     print("\n" + "="*40)
     print(" ERROR: Gradio Interface Cannot Be Launched ")
     print("Please check the errors above.")
     print("="*40 + "\n")
     if llm is None:
         print("Reason: Local LLM could not be initialized.")
         print("-> Review the 'ERROR initializing local LLM' messages above.")
         print("-> Check GPU/RAM resources, library installations (transformers, torch, accelerate, bitsandbytes if needed).")
         print("-> Verify Hugging Face token and model access permissions.")
     if calisanlar_df is None:
         print("Reason: Employee data (CSV) could not be loaded.")
         print("-> Review the 'ERROR loading employee CSV data' messages above.")
         print("-> Verify the file path:", CALISAN_BILGILERI_PATH)
         print("-> Ensure the CSV file exists and is readable.")
         print("-> If using Colab, ensure Google Drive is mounted correctly.")
     print("\nResolve the issues and restart the script.")
     print("-" * 40)

Hugging Face login successful (required for downloading gated models like Llama).

--- Initializing LOCAL LLM: 'meta-llama/Llama-3.2-3B-Instruct' ---
GPU Detected: Tesla T4
Using torch.float16 for model.

Loading Tokenizer...


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Tokenizer loaded successfully.

Loading Model 'meta-llama/Llama-3.2-3B-Instruct'...
This might take a while and require significant RAM/VRAM and Disk Space.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


------------------------------
Received Input: Sicil='32614', Dropdown='None', Message='bana rock şarkı öner'
Employee Found: Sicil=32614, Name=Eroğul Yorulmaz, Position=Satış Temsilcisi
Processing User Message: 'bana rock şarkı öner'
DEBUG: Sending general query to LOCAL LLM: 'bana rock şarkı öner'
DEBUG: Prompt sent to LLM:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Sen Kipaş şirketinin yardımsever bir İK asistanısın. Çalışan 'Eroğul Yorulmaz' (Pozisyon: Satış Temsilcisi) sana bir soru soruyor. Sorusuna kısa, profesyonel, nazik ve öz bir şekilde cevap ver. Eğer soru şirket politikaları, genel İK süreçleri veya işle ilgili genel konular hakkındaysa cevap ver. Çalışanın kişisel verilerini (maaş, izin vb.) ifşa etme, bu tür bilgiler istendiğinde 'Bu bilgiye erişimim yok, lütfen ilgili menüden kontrol edin veya İK departmanı ile iletişime geçin.' gibi bir yanıt ver.<|eot_id|><|start_header_id|>user<|end_header_id|>
bana rock şarkı öner<|eot_id|><|start_header_id|>assista

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


------------------------------
Received Input: Sicil='32614', Dropdown='None', Message='türkiyede nereleri gezebilirim'
Employee Found: Sicil=32614, Name=Eroğul Yorulmaz, Position=Satış Temsilcisi
Processing User Message: 'türkiyede nereleri gezebilirim'
DEBUG: Sending general query to LOCAL LLM: 'türkiyede nereleri gezebilirim'
DEBUG: Prompt sent to LLM:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Sen Kipaş şirketinin yardımsever bir İK asistanısın. Çalışan 'Eroğul Yorulmaz' (Pozisyon: Satış Temsilcisi) sana bir soru soruyor. Sorusuna kısa, profesyonel, nazik ve öz bir şekilde cevap ver. Eğer soru şirket politikaları, genel İK süreçleri veya işle ilgili genel konular hakkındaysa cevap ver. Çalışanın kişisel verilerini (maaş, izin vb.) ifşa etme, bu tür bilgiler istendiğinde 'Bu bilgiye erişimim yok, lütfen ilgili menüden kontrol edin veya İK departmanı ile iletişime geçin.' gibi bir yanıt ver.<|eot_id|><|start_header_id|>user<|end_header_id|>
türkiyede nereleri gezebil

KeyboardInterrupt: 